# **Easy-peasy use of Hospital 2552-10 Cost Report Data files**
-----
### 1. Load, format, filter all related or necessary HCRIS files
### 2. Combine 2552-10 cost report data files (2010 - 2019) into a single dataframe
### 4. Provide initial look at data and structure and prep for analysis

In [1]:
import pandas as pd
import warnings
from IPython.utils import io
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

## **Load and examine HOSPITAL10_PROVIDER_ID_INFO.CSV**

In [2]:
prvdr_info_df = pd.read_csv('hosp10-reports/HOSPITAL10_PROVIDER_ID_INFO.CSV', sep=',')
print('prvdr_info_df: (rows, columns) =', prvdr_info_df.shape)
rush_info = prvdr_info_df[prvdr_info_df['HOSP10_Name'] == 'RUSH UNIVERSITY MEDICAL CENTER']
rush_prvdr_num = rush_info['PROVIDER_NUMBER'].iloc[0]
print('Rush provider number:', rush_prvdr_num)
prvdr_info_df.head() # prvdr numbers with 5 digits actually have a leading zero

prvdr_info_df: (rows, columns) = (6675, 12)
Rush provider number: 140119


PROVIDER_NUMBER       FYB        FYE        STATUS  CTRL_TYPE  \
0            10001  1-Oct-18  30-Sep-19  As Submitted          9   
1            10005  1-Oct-18  30-Sep-19  As Submitted          9   
2            10006  1-Jul-18  30-Jun-19  As Submitted          4   
3            10007  1-Oct-18  30-Sep-19  As Submitted          9   
4            10008  1-Jan-18  31-Dec-18  As Submitted          4   

                       HOSP10_Name                  Street_Addr  \
0  SOUTHEAST HEALTH MEDICAL CENTER       1108 ROSS CLARK CIRCLE   
1  MARSHALL MEDICAL CENTER - SOUTH        2505 U.S. HIGHWAY 431   
2     NORTH ALABAMA MEDICAL CENTER          1701 VETERANS DRIVE   
3         MIZELL MEMORIAL HOSPITAL              702 MAIN STREET   
4      CRENSHAW COMMUNITY HOSPITAL  CRENSHAW COMMUNITY HOSPITAL   

                PO_Box      City State Zip_Code      County  
0                 6987    DOTHAN    AL    36301     HOUSTON  
1                  NaN      BOAZ    AL   35957-    MARSHALL  
2                  818  FLORENCE    AL    35630  LAUDERDALE  
3                  429       OPP    AL   36462-   COVINGTON  
4  101 HOSPITAL CIRCLE   LUVERNE    AL    36049    CRENSHAW

## **Load, format, and examine 'hosp10_oracle_counts.sas7bdat' file**

In [3]:
oracle_counts_df = pd.read_sas('hosp10-sas/hosp10_oracle_counts.sas7bdat')
for col in oracle_counts_df:
    if isinstance(oracle_counts_df[col][0], bytes):
        oracle_counts_df[col] = oracle_counts_df[col].str.decode("utf8")  # or any other encoding
print('oracle_counts_df: (rows, columns) =', oracle_counts_df.shape)
oracle_counts_df.head(10)

oracle_counts_df: (rows, columns) = (10, 17)


fy  rpt_ora  rpt_sas  rpt_merged  rpt_dupes  alpha_sample  nmrc_sample  \
0  2010.0   2323.0   2323.0      2323.0        0.0        2294.0       2323.0   
1  2011.0   6150.0   6150.0      6150.0        0.0        6080.0       6150.0   
2  2012.0   6227.0   6227.0      6227.0        0.0        6129.0       6227.0   
3  2013.0   6248.0   6248.0      6248.0        0.0        6187.0       6248.0   
4  2014.0   6249.0   6249.0      6249.0        0.0        6180.0       6249.0   
5  2015.0   6257.0   6257.0      6257.0        0.0        6178.0       6257.0   
6  2016.0   6211.0   6211.0      6211.0        0.0        6126.0       6211.0   
7  2017.0   6166.0   6166.0      6166.0        0.0        6089.0       6166.0   
8  2018.0   6121.0   6121.0      6121.0        0.0        6072.0       6121.0   
9  2019.0    989.0    989.0       989.0        0.0         985.0        989.0   

   rollup_sample  alpha_ora  alpha_sas    nmrc_ora    nmrc_sas  rollup_ora  \
0         2294.0  1352988.0  1352988.0   7408064.0   7408064.0    751400.0   
1         6070.0  3545739.0  3545739.0  19353253.0  19353253.0   2001382.0   
2         6129.0  3608395.0  3608395.0  19698554.0  19698554.0   2023167.0   
3         6186.0  3678665.0  3678665.0  19923117.0  19923117.0   2052566.0   
4         6178.0  3688743.0  3688743.0  19849304.0  19849304.0   2050312.0   
5         6175.0  3693407.0  3693407.0  19737033.0  19737033.0   2036733.0   
6         6122.0  3696393.0  3696393.0  19584812.0  19584812.0   2019498.0   
7         6083.0  3684286.0  3684286.0  19403209.0  19403209.0   2873071.0   
8         6067.0  3661249.0  3661249.0  19282180.0  19282180.0   2008962.0   
9          985.0   598491.0   598491.0   3321089.0   3321089.0    342234.0   

   rollup_sas  recent fy_str  diffs  
0    751400.0    25.0   2010    0.0  
1   2001382.0    45.0   2011    0.0  
2   2023167.0    72.0   2012    0.0  
3   2052566.0   109.0   2013    0.0  
4   2050312.0   138.0   2014    0.0  
5   2036733.0   159.0   2015    0.0  
6   2019498.0   479.0   2016    0.0  
7   2873071.0  1286.0   2017    0.0  
8   2008962.0   951.0   2018    0.0  
9    342234.0   906.0   2019    0.0

## **Load, format, and examine 'prds_hosp10_yr' sas7bdat files**
## Concatenate files into a single dataframe (main_df) and join with the crosswalk file

In [4]:
with io.capture_output() as captured: prds_2010_df = pd.read_sas('hosp10-sas/prds_hosp10_yr2010.sas7bdat')
print('prds_hosp10_yr2010: (rows, columns) =', prds_2010_df.shape)
with io.capture_output() as captured: prds_2011_df = pd.read_sas('hosp10-sas/prds_hosp10_yr2011.sas7bdat')
print('prds_hosp10_yr2011: (rows, columns) =', prds_2011_df.shape)
with io.capture_output() as captured: prds_2012_df = pd.read_sas('hosp10-sas/prds_hosp10_yr2012.sas7bdat')
print('prds_hosp10_yr2012: (rows, columns) =', prds_2012_df.shape)
with io.capture_output() as captured: prds_2013_df = pd.read_sas('hosp10-sas/prds_hosp10_yr2013.sas7bdat')
print('prds_hosp10_yr2013: (rows, columns) =', prds_2013_df.shape)
with io.capture_output() as captured: prds_2014_df = pd.read_sas('hosp10-sas/prds_hosp10_yr2014.sas7bdat')
print('prds_hosp10_yr2014: (rows, columns) =', prds_2014_df.shape)
with io.capture_output() as captured: prds_2015_df = pd.read_sas('hosp10-sas/prds_hosp10_yr2015.sas7bdat')
print('prds_hosp10_yr2015: (rows, columns) =', prds_2015_df.shape)
with io.capture_output() as captured: prds_2016_df = pd.read_sas('hosp10-sas/prds_hosp10_yr2016.sas7bdat')
print('prds_hosp10_yr2016: (rows, columns) =', prds_2016_df.shape)
with io.capture_output() as captured: prds_2017_df = pd.read_sas('hosp10-sas/prds_hosp10_yr2017.sas7bdat')
print('prds_hosp10_yr2017: (rows, columns) =', prds_2017_df.shape)
with io.capture_output() as captured: prds_2018_df = pd.read_sas('hosp10-sas/prds_hosp10_yr2018.sas7bdat')
print('prds_hosp10_yr2018: (rows, columns) =', prds_2018_df.shape)
with io.capture_output() as captured: prds_2019_df = pd.read_sas('hosp10-sas/prds_hosp10_yr2019.sas7bdat')
print('prds_hosp10_yr2019: (rows, columns) =', prds_2019_df.shape)

prds_hosp10_yr2010: (rows, columns) = (2323, 5581)
prds_hosp10_yr2011: (rows, columns) = (6150, 5581)
prds_hosp10_yr2012: (rows, columns) = (6227, 5581)
prds_hosp10_yr2013: (rows, columns) = (6248, 5581)
prds_hosp10_yr2014: (rows, columns) = (6249, 5581)
prds_hosp10_yr2015: (rows, columns) = (6257, 5581)
prds_hosp10_yr2016: (rows, columns) = (6211, 5581)
prds_hosp10_yr2017: (rows, columns) = (6166, 5581)
prds_hosp10_yr2018: (rows, columns) = (6121, 5581)
prds_hosp10_yr2019: (rows, columns) = (989, 5581)


In [5]:
main_df = pd.concat([prds_2019_df, prds_2018_df, prds_2017_df, prds_2016_df, prds_2015_df, 
                     prds_2014_df, prds_2013_df, prds_2012_df, prds_2011_df, prds_2010_df,
                     ], ignore_index=True)

main_df = main_df.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
main_df = main_df.dropna(axis=1, how='all')

In [6]:
to_replace = ['rpt_rec_num', 'prvdr_num', 'fi_num', 'rpt_stus_cd', 'fi_creat_dt', 
              'fy_bgn_dt', 'fy_end_dt', 'util_cd', 'trnsmtl_num', 'state', 
              'st_cty_cd', 'census', 'region', 'proc_dt', 'msa']

replacement = ['RPT_REC_NUM', 'PRVDR_NUM', 'FI_NUM', 'RPT_STUS_CD', 'FI_CREAT_DT', 
               'FY_BGN_DT', 'FY_END_DT', 'UTIL_CODE', 'TRNSMTL_NUM', 'STATE', 
               'ST_CTY_CD', 'CENSUS', 'REGION', 'PROC_DT', 'MSA']

main_df.rename(columns = {to_replace[0]: replacement[0], to_replace[1]: replacement[1],
                           to_replace[2]: replacement[2], to_replace[3]: replacement[3],
                           to_replace[4]: replacement[4], to_replace[5]: replacement[5],
                           to_replace[6]: replacement[6], to_replace[7]: replacement[7],
                           to_replace[8]: replacement[8], to_replace[9]: replacement[9],
                           to_replace[10]: replacement[10], to_replace[11]: replacement[11],
                           to_replace[12]: replacement[12], to_replace[13]: replacement[13],
                           to_replace[14]: replacement[14] }, inplace = True)

main_df.drop(['_NAME_'], axis=1, inplace=True)

In [7]:
crosswalk_df = pd.read_csv('crosswalk/2552-10_SAS_FILE_RECORD_LAYOUT_AND_CROSSWALK_TO_96.csv', sep=',')
crosswalk_labels = crosswalk_df['10_FIELD_NAME'].tolist()
crosswalk_labels = [str(x).strip(' ') for x in crosswalk_labels]
crosswalk_df['10_FIELD_NAME'] = crosswalk_labels

In [8]:
main_df_col_labels = list(main_df)
main_df_col_labels = [str(x).strip(' ') for x in main_df_col_labels]
print('main_df with NaN columns removed: (rows, columns) =', main_df.shape)

print('\nNumber of labels in the crosswalk:', len(crosswalk_labels))
print('Number of unique labels in the crosswalk:', len(list(set(crosswalk_labels))))

print('\nNumber of labels in the main_df:', len(main_df_col_labels))
print('Number of unique labels in the main_df:', len(list(set(main_df_col_labels))))

shared_labels = list(set(main_df_col_labels) & set(crosswalk_labels))
print('\nNumber of labels shared between the main dataframe and the crosswalk:', len(shared_labels))

#ct = 0
#for lab in main_df_col_labels:
#    if lab in crosswalk_labels:
#        pass
#    else:
#        ct += 1
#        print('\nLabels in main_df but not in crosswalk:', [lab])
#print(ct)

#ct = 0
#for lab in crosswalk_labels:
#    if lab in main_df_col_labels:
#        pass
#    else:
#        ct += 1
#        print('\nLabels in crosswalk but not in main_df:', [lab])
#print(ct)

main_df.head()

main_df with NaN columns removed: (rows, columns) = (52941, 3928)

Number of labels in the crosswalk: 5556
Number of unique labels in the crosswalk: 5502

Number of labels in the main_df: 3928
Number of unique labels in the main_df: 3928

Number of labels shared between the main dataframe and the crosswalk: 3810


RPT_REC_NUM PRVDR_NUM FI_NUM RPT_STUS_CD FI_CREAT_DT  FY_BGN_DT  FY_END_DT  \
0     649071.0    040018  05901           1  2019-06-14 2018-11-01 2018-12-31   
1     649213.0    150176  08001           1  2019-06-14 2019-01-01 2019-04-07   
2     649407.0    100044  09001           1  2019-06-18 2018-10-01 2018-12-31   
3     649741.0    200019  14011           1  2019-06-15 2018-10-01 2018-12-31   
4     649882.0    084005  12001           1  2019-06-24 2018-11-01 2018-12-31   

  UTIL_CODE TRNSMTL_NUM STATE ST_CTY_CD  CENSUS  REGION    PROC_DT   MSA  \
0         F           M    04     04160     7.0     6.0 2019-06-17  2720   
1         F           M   NaN       NaN     NaN     NaN 2019-06-18   NaN   
2         F           M    10     10420     5.0     4.0 2019-06-19  2710   
3         F           M    20     20150     1.0     1.0 2019-06-21  6403   
4         F           M   NaN       NaN     NaN     NaN 2019-06-24   NaN   

   A7_1_C3_1  A7_1_C3_4  A7_1_C3_7  A_C1_111  A_C1_114  A_C1_4201  A_C1_4501  \
0        NaN        NaN        NaN       NaN       NaN        NaN        NaN   
1        NaN        NaN        NaN       NaN       NaN        NaN        NaN   
2        NaN        NaN        NaN       NaN       NaN        NaN        NaN   
3        NaN        NaN        NaN       NaN       NaN        NaN        NaN   
4        NaN        NaN        NaN       NaN       NaN        NaN        NaN   

   A_C2_111  A_C2_114  A_C2_4201  A_C2_4501  A_C2_61  A_C6_100  A_C6_111  \
0       NaN       NaN        NaN        NaN      NaN       NaN       NaN   
1       NaN       NaN        NaN        NaN      NaN       NaN       NaN   
2       NaN       NaN        NaN        NaN      NaN       NaN       NaN   
3       NaN       NaN        NaN        NaN      NaN       NaN       NaN   
4       NaN       NaN        NaN        NaN      NaN       NaN       NaN   

   A_C6_42  A_C6_4201  A_C6_4501  A_C6_61  B_1_C0_111  B_1_C0_4201  \
0      NaN        NaN        NaN      NaN         NaN          NaN   
1      NaN        NaN        NaN      NaN         NaN          NaN   
2      NaN        NaN        NaN      NaN         NaN          NaN   
3      NaN        NaN        NaN      NaN         NaN          NaN   
4      NaN        NaN        NaN      NaN         NaN          NaN   

   B_1_C0_4501  B_1_C0_61  B_1_C21_100  B_1_C21_108  B_1_C21_109  B_1_C21_42  \
0          NaN        NaN          NaN          NaN          NaN         NaN   
1          NaN        NaN          NaN          NaN          NaN         NaN   
2          NaN        NaN          NaN          NaN          NaN         NaN   
3          NaN        NaN          NaN          NaN          NaN         NaN   
4          NaN        NaN          NaN          NaN          NaN         NaN   

   B_1_C21_45  B_1_C21_46  B_1_C21_94  B_1_C22_100  B_1_C22_108  B_1_C22_109  \
0         NaN         NaN         NaN          NaN          NaN          NaN   
1         NaN         NaN         NaN          NaN          NaN          NaN   
2         NaN         NaN         NaN          NaN          NaN          NaN   
3         NaN         NaN         NaN          NaN          NaN          NaN   
4         NaN         NaN         NaN          NaN          NaN          NaN   

   B_1_C22_110  B_1_C22_111  B_1_C22_42  B_1_C22_45  B_1_C22_46  B_1_C22_94  \
0          NaN          NaN         NaN         NaN         NaN         NaN   
1          NaN          NaN         NaN         NaN         NaN         NaN   
2          NaN          NaN         NaN         NaN         NaN         NaN   
3          NaN          NaN         NaN         NaN         NaN         NaN   
4          NaN          NaN         NaN         NaN         NaN         NaN   

   B_1_C26_111  B_1_C26_4201  B_1_C26_4501  B_1_C26_61  B_2_C26_111  \
0          NaN           NaN           NaN         NaN          NaN   
1          NaN           NaN           NaN         NaN          NaN   
2          NaN           NaN           NaN        

In [9]:
main_df2 = main_df[main_df.columns[main_df.columns.isin(shared_labels)]]

FIELD_DESCRIPTION = []
TYPE = []
SUBTYPE = []
#DATA_TYPE = []

col_labels = list(main_df2)
for lab in col_labels:
    df_sub = crosswalk_df[crosswalk_df['10_FIELD_NAME_CORRECTED'] == lab]
    FIELD_DESCRIPTION.append(df_sub['FIELD DESCRIPTION'].iloc[0])
    TYPE.append(df_sub['TYPE'].iloc[0])
    SUBTYPE.append(df_sub['SUBTYPE'].iloc[0])
    #DATA_TYPE.append(df_sub['DATA_TYPE'].iloc[0])

df2 = pd.DataFrame([col_labels, FIELD_DESCRIPTION, TYPE, SUBTYPE], columns=col_labels)
main_df2 = pd.concat([df2, main_df2])


main_df2.columns=pd.MultiIndex.from_arrays(main_df2.iloc[0:4].values)
main_df2 = main_df2.iloc[4:]

In [47]:
main_df3 = main_df2.loc[:, ~main_df2.columns.get_level_values(1).isin({float('NaN')})]
print(main_df3.shape)
#main_df3.head(10)

(52941, 2673)


In [57]:
#print(list(set(main_df3.columns.get_level_values(2))))
#print(len(list(set(main_df3.columns.get_level_values(2)))))
#main_df4 = main_df3.iloc[:, main_df3.columns.get_level_values(2) == 'BED DAYS AVAILABLE']
main_df4 = main_df3.dropna(axis=1, thresh=50000)
print(main_df4.shape)
main_df4.head()

(52941, 92)


RPT_REC_NUM                           PRVDR_NUM  \
                 Report Record Number           Hospital Provider Number    
  HOSPITAL IDENTIFICATION INFORMATION HOSPITAL IDENTIFICATION INFORMATION   
                                  NaN                                 NaN   
0                              649071                              040018   
1                              649213                              150176   
2                              649407                              100044   
3                              649741                              200019   
4                              649882                              084005   

                               FI_NUM                         RPT_STUS_CD  \
           Fiscal Intermediary Number                  Report Status Code   
  HOSPITAL IDENTIFICATION INFORMATION HOSPITAL IDENTIFICATION INFORMATION   
                                  NaN                                 NaN   
0                               05901                                   1   
1                               08001                                   1   
2                               09001                                   1   
3                               14011                                   1   
4                               12001                                   1   

                          FI_CREAT_DT                           FY_BGN_DT  \
  Fiscal Intermediary/MAC Create Date             Fiscal Year Begin Date    
  HOSPITAL IDENTIFICATION INFORMATION HOSPITAL IDENTIFICATION INFORMATION   
                                  NaN                                 NaN   
0                 2019-06-14 00:00:00                 2018-11-01 00:00:00   
1                 2019-06-14 00:00:00                 2019-01-01 00:00:00   
2                 2019-06-18 00:00:00                 2018-10-01 00:00:00   
3                 2019-06-15 00:00:00                 2018-10-01 00:00:00   
4                 2019-06-24 00:00:00                 2018-11-01 00:00:00   

                            FY_END_DT                           UTIL_CODE  \
                Fiscal Year End Date             Utilization Code (L/N/F)   
  HOSPITAL IDENTIFICATION INFORMATION HOSPITAL IDENTIFICATION INFORMATION   
                                  NaN                                 NaN   
0                 2018-12-31 00:00:00                                   F   
1                 2019-04-07 00:00:00                                   F   
2                 2018-12-31 00:00:00                                   F   
3                 2018-12-31 00:00:00                                   F   
4                 2018-12-31 00:00:00                                   F   

                          TRNSMTL_NUM                             A7_3_C15_3  \
                     NOT IN THIS FILE                                  Total   
  HOSPITAL IDENTIFICATION INFORMATION RECONCILIATION OF CAPITAL COST CENTERS   
                                  NaN                      Column 15 (Total)   
0                                   M                                 134927   
1                                   M                            1.78405e+06   
2                                   M                            1.51405e+07   
3                                   M                            6.95648e+06   
4                                   M                                 183618   

                               A7_3_C9_3     A8_C2_50  \
                                   Total        Total   
  RECONCILIATION OF CAPITAL COST CENTERS  ADJUSTMENTS   
                 Column 9 (Depreciation)          NaN   
0                                 132800       163629   
1                                 339180 -2.64467e+06   
2                            1.07131e+07 -1.01191e+07   
3                            4.68861e+06 -1.33237e+07   
4                                 166868        34887   

               

In [65]:
main_df5 = main_df4[main_df4.columns.drop(list(main_df4.filter(regex='(Y/N)')))]
main_df5 = main_df5[main_df5.columns.drop(list(main_df5.filter(regex='costs?')))]
main_df5 = main_df5[main_df5.columns.drop(list(main_df5.filter(regex='NOT IN THIS FILE')))]
main_df5 = main_df5[main_df5.columns.drop(list(main_df5.filter(regex='Report Status Code')))]
main_df5 = main_df5[main_df5.columns.drop(list(main_df5.filter(regex='Urban/Rural Indicator at beginning')))]
print(main_df5.shape)
main_df5.head()
main_df5.to_pickle('dataframe_data/main_df.pkl')

(52941, 75)
